In [12]:
# First, we add all the packages we need to the system.
# This is what enables us to run all the code below.

import os
from dotenv import load_dotenv

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import Agent, ChatCompletionAgent, GroupChatOrchestration, OrchestrationHandoffs, RoundRobinGroupChatManager
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.agents.runtime import InProcessRuntime

In [13]:
# Here, we connect to the Azure AI Foundry, and get the necessary environment variables
# including the project endpoint, model deployment name, and tenant ID.

load_dotenv()
endpoint = os.getenv("ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")
api_key = os.getenv("API_KEY")
api_version = os.getenv("API_VERSION")

In [14]:
# This block of code connects to the MCP Server
async with MCPStdioPlugin(
    name="InventoryManagementMCPPlugin",
    description="Inventory Management MCP Plugin",
    command="python",
    args=["server.py"],
) as inventory_plugin:
    pass
 
chat_service = AzureChatCompletion(
    endpoint=endpoint,
    deployment_name=model_deployment,
    api_version=api_version,
    api_key=api_key
)

In [16]:
def get_agents() -> tuple[list[Agent], OrchestrationHandoffs]:
    # Retorna uma lista de agentes
    
    inventory_agent = ChatCompletionAgent(
        name="InventoryAgent",
        description="Um agente que possui conhecimento sobre o inventário.",
        instructions="Realize pesquisas e forneça informações sobre o inventário, incluindo níveis de estoque e detalhes dos itens.",
        service=chat_service,
        plugins=[inventory_plugin],
    )
    
    accounting_agent = ChatCompletionAgent(
        name="AccountingAgent",
        description="Um agente que possui conhecimento sobre tarefas de contabilidade.",
        instructions="Receba relatórios do status do inventário e prepare uma lista de itens para comprar, considerando um orçamento de R$1000.",
        service=chat_service,
        plugins=[inventory_plugin],
    )
    
    inventory_purchasing_agent = ChatCompletionAgent(
        name="InventoryPurchasingAgent",
        description="Um agente capaz de comprar itens do inventário.",
        instructions="Realize tarefas de compra de itens do inventário e compre os itens recomendados pelo agente de contabilidade. Utilize ferramentas para fazer pedidos e depois confirme o pedido.",
        service=chat_service,
        plugins=[inventory_plugin],
    )
    
    return [inventory_agent, accounting_agent, inventory_purchasing_agent]

In [17]:
def agent_response_callback(message: ChatMessageContent) -> None:
    """Observer function to print the messages from the agents."""
    if message.content:
        print(f"🤖{message.name}\n{message.content}")

In [18]:
await inventory_plugin.connect()

# 1. Crie uma orquestração de chat em grupo com um gerente round robin
agents = get_agents()
group_chat_orchestration = GroupChatOrchestration(
    members=agents,
    # max_rounds é ímpar, para que o escritor tenha a última rodada
    manager=RoundRobinGroupChatManager(max_rounds=5),
    agent_response_callback=agent_response_callback,
)

# 2. Crie um runtime e inicie-o
runtime = InProcessRuntime()
runtime.start()

# 3. Invoque a orquestração com uma tarefa e o runtime
orchestration_result = await group_chat_orchestration.invoke(
    task="""
    Use o InventoryAgent para pesquisar todos os produtos no inventário que estão com baixo estoque e prepare um relatório para o AccountingAgent.
    Certifique-se de que o AccountingAgent retorne uma lista de itens para comprar, considerando um orçamento de R$1000.
    Envie a lista de itens para comprar ao InventoryPurchasingAgent e atualize o estoque com o que o AccountingAgent sugeriu.
    Não pergunte se o estoque deve ser atualizado, apenas faça.
    Ao finalizar, utilize as ferramentas para recuperar as novas unidades em estoque dos produtos no inventário com o InventoryAgent, indicando os níveis de estoque antes e depois.""",
    runtime=runtime,
)

# 4. Aguarde os resultados
value = await orchestration_result.get()
print(f"***** Resultado *****\n{value}")

# 5. Pare o runtime após a conclusão da invocação
await runtime.stop_when_idle()
await inventory_plugin.close() if inventory_plugin else None

🤖InventoryAgent
Produtos considerados de baixo estoque, com base na regra de recomendação (estoque < 10 AND vendas semanais > 15):

- Konbu: Estoque 0, Vendas Semanais 0 (não recomendado, vendas semanais 0)
- Alice Mutton: Estoque 0, Vendas Semanais 0 (não recomendado, vendas semanais 0)

Nenhum produto atende à condição de baixo estoque com alta demanda para recomendação de reposição.

No entanto, há produtos com estoque baixo (menos que 20) e com algum nível de vendas, que poderiam ser considerados para reposição na análise do AccountingAgent.

Vou preparar um relatório com os produtos que têm estoque baixo (menos ou igual a 20 unidades), listando estoque atual e vendas semanais, para o AccountingAgent avaliar e retornar os itens para comprar dentro do orçamento de R$1000.
🤖InventoryAgent
Produtos considerados de baixo estoque, com base na regra de recomendação (estoque < 10 AND vendas semanais > 15):

- Konbu: Estoque 0, Vendas Semanais 0 (não recomendado, vendas semanais 0)
- Alice